In [12]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Dec 17 23:27:05 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P0              27W /  70W |   2993MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
pip install transformers datasets torch scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [12]:
from datasets import load_dataset

# Load AG News - text classification 4 classes
ag_news = load_dataset("ag_news")

# Load SST-2 - sentiment analysis - binary classification
sst2 = load_dataset("glue", "sst2")

print(ag_news["train"][0])  # Print example from AG News
print(sst2["train"][0])     # Print example from SST-2

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'label': 2}
{'sentence': 'hide new secretions from the parental units ', 'label': 0, 'idx': 0}


In [13]:
print(sst2["train"])

Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 67349
})


In [3]:
pip install textattack

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 35.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 47.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.7/445.7 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.5/776.5 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import textattack
from textattack.attack_recipes import TextFoolerJin2019
from textattack.models.wrappers import HuggingFaceModelWrapper
from transformers import AutoModelForSequenceClassification

# Load model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

# Define Attack
attack = TextFoolerJin2019.build(model_wrapper)

# Example input
input_text = "The economy is looking good for the next quarter."
label = 2  # Label for "Business"

# Generate adversarial example
attack_result = attack.attack(textattack.shared.AttackedText(input_text), label)
print("Original Text:", input_text)
print("Adversarial Example:", attack_result.perturbed_text())

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set i

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


NameError: name 'AutoTokenizer' is not defined

In [13]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from datasets import load_dataset

# Step 1: Load Pretrained Model and Tokenizer
MODEL_NAME = "bert-base-uncased"  # Choose any Hugging Face model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

# Device setup (CUDA if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Step 2: Load and Preprocess Dataset
def preprocess_data(examples):
    return tokenizer(examples['sentence'], truncation=True, padding='max_length', max_length=128)

# Load a sample dataset (SST-2 binary classification)
dataset = load_dataset("sst2")
train_dataset = dataset['train'].map(preprocess_data, batched=True)
test_dataset = dataset['validation'].map(preprocess_data, batched=True)

# Convert to PyTorch-compatible format
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16)

# Step 3: Fine-tune the Model on Clean Data
optimizer = AdamW(model.parameters(), lr=5e-5)
criterion = nn.CrossEntropyLoss()

print("Training on clean data...")
model.train()
for epoch in range(2):  # Train for 2 epochs
    for batch in train_dataloader:
        input_ids = torch.tensor(batch['input_ids']).to(device)
        attention_mask = torch.tensor(batch['attention_mask']).to(device)
        labels = torch.tensor(batch['label']).to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1} completed.")

model.save_pretrained("saved_model")
tokenizer.save_pretrained("saved_model")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/872 [00:00<?, ? examples/s]

  0%|          | 0/50 [11:59<?, ?it/s]
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-13-ce5b9b951285>:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(batch['input_ids']).to(device)
<ipython-input-13-ce5b9b951285>:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(batch['attention_mask']).to(device)
<ipython-input-13-ce5b9b951285>:42: UserWarning: To copy const

Training on clean data...
Epoch 1 completed.
Epoch 2 completed.


('saved_model/tokenizer_config.json',
 'saved_model/special_tokens_map.json',
 'saved_model/vocab.txt',
 'saved_model/added_tokens.json',
 'saved_model/tokenizer.json')

In [40]:
# Step 4: Generate Adversarial Examples using TextAttack
print("Generating adversarial examples...")

from textattack.attack_recipes import TextFoolerJin2019
from textattack.models.wrappers import HuggingFaceModelWrapper
#from textattack.attacks import Attack
from textattack import Attacker, AttackArgs
from textattack.datasets import Dataset as TextAttackDataset
#from transformers import AutoModelForSequenceClassification, AutoTokenizer
import nltk
nltk.download('averaged_perceptron_tagger_eng')

# Load the model
#model = AutoModelForSequenceClassification.from_pretrained("saved_model")
#tokenizer = AutoTokenizer.from_pretrained("saved_model")

# Wrap the model for TextAttack
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

# Define the adversarial attack (TextFooler)
attack = TextFoolerJin2019.build(model_wrapper)
print(test_dataset[0])
# Convert HuggingFace dataset to TextAttack format
test_samples = []
for ex in test_dataset:
    text = tokenizer.decode(ex['input_ids'], skip_special_tokens=True)  # Decode input_ids to text
    label = ex['label'].item()
    #test_samples.append({'text': text, 'label': label})

    test_samples.append((text, label))

# Now, create a TextAttack dataset
textattack_dataset = TextAttackDataset(test_samples)
print(test_samples[0])
# Verify the dataset
#print(f"first element: {textattack_dataset[0][0]['text']}")  # Check the first element
# Set attack arguments
attack_args = AttackArgs(
    num_examples=50,  # Generate adversarial examples for 50 samples
    log_to_csv="adversarial_examples.csv",  # Save the adversarial examples
    disable_stdout=False,
)
# Run the attack
attacker = Attacker(attack, textattack_dataset, attack_args)
attack_results = attacker.attack_dataset()


[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Generating adversarial examples...
{'label': tensor(1), 'input_ids': tensor([  101,  2009,  1005,  1055,  1037, 11951,  1998,  2411, 12473,  4990,
         1012,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,   

textattack: Logging to CSV at path adversarial_examples.csv


("it ' s a charming and often affecting journey.", 1)
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (match

[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   2%|▏         | 1/50 [00:15<12:34, 15.40s/it]

...

--------------------------------------------- Result 48 ---------------------------------------------
[[1 (100%)]] --> [[0 (99%)]]

a subtle and [[well]] - [[crafted]] ( for the most part ) chiller.

a subtle and [[too]] - [[fabricated]] ( for the most part ) chiller.




[Succeeded / Failed / Skipped / Total] 41 / 6 / 2 / 49:  98%|█████████▊| 49/50 [02:12<00:02,  2.71s/it]

--------------------------------------------- Result 49 ---------------------------------------------
[[1 (99%)]] --> [[0 (92%)]]

has a [[lot]] of the virtues of eastwood at his best.

has a [[shipments]] of the virtues of eastwood at his best.




[Succeeded / Failed / Skipped / Total] 42 / 6 / 2 / 50: 100%|██████████| 50/50 [02:14<00:00,  2.69s/it]

--------------------------------------------- Result 50 ---------------------------------------------
[[0 (99%)]] --> [[1 (56%)]]

it ' s [[hampered]] by a lifetime - channel kind of plot and a lead actress who is out of her depth.

it ' s [[bothered]] by a lifetime - channel kind of plot and a lead actress who is out of her depth.



+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 42    |
| Number of failed attacks:     | 6     |
| Number of skipped attacks:    | 2     |
| Original accuracy:            | 96.0% |
| Accuracy under attack:        | 12.0% |
| Attack success rate:          | 87.5% |
| Average perturbed word %:     | 14.7% |
| Average num. words per input: | 16.6  |
| Avg num queries:              | 86.67 |
+-------------------------------+-------+


In [41]:
import pandas as pd
from sklearn.metrics import accuracy_score
import torch

# Load the adversarial examples
adv_examples = pd.read_csv("adversarial_examples.csv")

# Extract original and adversarial texts
original_texts = adv_examples['original_text'].tolist()
adversarial_texts = adv_examples['perturbed_text'].tolist()
labels = adv_examples['ground_truth_output'].tolist()

# Function to evaluate a list of inputs
def evaluate_model_on_texts(texts, labels, model, tokenizer, device):
    model.eval()
    predictions = []
    for text in texts:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            predicted = torch.argmax(outputs.logits, dim=1).item()
            predictions.append(predicted)
    return accuracy_score(labels, predictions), predictions

# Baseline Evaluation on Adversarial Examples (Before Robust Training)
print("Evaluating on adversarial examples BEFORE robust training...")
adv_accuracy_before, _ = evaluate_model_on_texts(adversarial_texts, labels, model, tokenizer, device)
print(f"Accuracy on adversarial examples (Before): {adv_accuracy_before * 100:.2f}%")


Evaluating on adversarial examples BEFORE robust training...
Accuracy on adversarial examples (Before): 58.00%


In [46]:
# Step 5: Combine Clean and Adversarial Data for Robust Training
print("Preparing data for robust training...")
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset

# Load adversarial examples from CSV
adv_examples = pd.read_csv("adversarial_examples.csv")

# Create a combined dataset: clean + adversarial
combined_data = []
for idx, row in adv_examples.iterrows():
    combined_data.append({"text": row["original_text"], "label": row["ground_truth_output"]})
    combined_data.append({"text": row["perturbed_text"], "label": row["ground_truth_output"]})

class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data[idx]["text"]
        label = self.data[idx]["label"]

        # Tokenize the text
        tokenized = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        )

        # Squeeze batch dimension and return as dictionary
        return {
            "input_ids": tokenized["input_ids"].squeeze(0),
            "attention_mask": tokenized["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.long),
        }

# Step 2: Create PyTorch dataset and dataloader
combined_dataset = CustomDataset(combined_data, tokenizer)
robust_train_dataloader = DataLoader(combined_dataset, batch_size=16, shuffle=True)

# Step 6: Retrain the Model on Combined Data (Clean + Adversarial)
print("Training on clean and adversarial data...")
model.train()
for epoch in range(2):  # Robust training for 2 epochs
    for batch in robust_train_dataloader:
        input_ids = torch.tensor(batch['input_ids']).to(device)
        attention_mask = torch.tensor(batch['attention_mask']).to(device)
        labels = torch.tensor(batch['label']).to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1} of robust training completed.")

# Step 7: Evaluate the Model
print("Evaluating the model...")
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for batch in test_dataloader:
        input_ids = torch.tensor(batch['input_ids']).to(device)
        attention_mask = torch.tensor(batch['attention_mask']).to(device)
        labels = torch.tensor(batch['label']).to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, predicted = torch.max(outputs.logits, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Preparing data for robust training...
Training on clean and adversarial data...


<ipython-input-46-205457b6a5f0>:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(batch['input_ids']).to(device)
<ipython-input-46-205457b6a5f0>:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(batch['attention_mask']).to(device)
<ipython-input-46-205457b6a5f0>:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch['label']).to(device)


Epoch 1 of robust training completed.
Epoch 2 of robust training completed.
Evaluating the model...


<ipython-input-46-205457b6a5f0>:74: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(batch['input_ids']).to(device)
<ipython-input-46-205457b6a5f0>:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(batch['attention_mask']).to(device)
<ipython-input-46-205457b6a5f0>:76: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch['label']).to(device)


Test Accuracy: 89.11%


In [47]:
from pathlib import Path

# Define the output directory
output_dir = "./trained_model"

# Create the directory if it doesn't exist
Path(output_dir).mkdir(parents=True, exist_ok=True)

# Save the trained model and tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Model and tokenizer saved to {output_dir}")


Model and tokenizer saved to ./trained_model


In [ ]:
# Evaluate on Clean Test Data
print("Evaluating on clean test data...")
test_texts = [ex['sentence'] for ex in test_dataset]
test_labels = [ex['label'] for ex in test_dataset]
clean_accuracy, _ = evaluate_model_on_texts(test_texts, test_labels, model, tokenizer, device)
print(f"Accuracy on clean test data: {clean_accuracy * 100:.2f}%")


In [48]:
# Evaluation on Adversarial Examples (After Robust Training)
print("Evaluating on adversarial examples AFTER robust training...")
adv_accuracy_after, _ = evaluate_model_on_texts(adversarial_texts, labels, model, tokenizer, device)
print(f"Accuracy on adversarial examples (After): {adv_accuracy_after * 100:.2f}%")


Evaluating on adversarial examples AFTER robust training...


ValueError: Found input variables with inconsistent numbers of samples: [8, 50]

In [49]:
# Load adversarial examples from CSV
adv_examples = pd.read_csv("adversarial_examples.csv")

# Prepare adversarial texts and labels
adv_texts = adv_examples["perturbed_text"].tolist()
adv_labels = adv_examples["ground_truth_output"].tolist()

# Evaluate the model on adversarial examples
adv_accuracy, adv_predictions = evaluate_model_on_texts(adv_texts, adv_labels, model, tokenizer, device)

print(f"Accuracy on adversarial examples after training: {adv_accuracy * 100:.2f}%")


Accuracy on adversarial examples after training: 98.00%
